In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import datetime as dt

%pip install plotly --upgrade
%pip install nbformat --upgrade



# Feature todo
"""
1. Load the data from CSV files
2. Display the first few rows of each dataset
3. (Future) Create visualizations using Plotly
3.1 Chart for satisfaction over time (patients_data)
3.2 relationship between time spent, services and satisfaction (patients_data, patients_data)
3.4 relationship between staff service, morale and patient satisfaction (patients_data)
4. probability of staff absence based on morale and workload (staff_data, staff_schedule)
5. join staff and staff schedule data to analyze patterns
"""
patients_data = pd.read_csv('../data/raw/patients.csv')
services_data = pd.read_csv('../data/raw/services_weekly.csv')
staff_schedule = pd.read_csv('../data/raw/staff_schedule.csv')
staff_data = pd.read_csv('../data/raw/staff.csv')


patients_data.head()



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


,patient_id,name,age,arrival_date,departure_date,service,satisfaction
0,PAT-09484753,Richard Rodriguez,24,2025-03-16,2025-03-22,surgery,61
1,PAT-f0644084,Shannon Walker,6,2025-12-13,2025-12-14,surgery,83
2,PAT-ac6162e4,Julia Torres,24,2025-06-29,2025-07-05,general_medicine,83
3,PAT-3dda2bb5,Crystal Johnson,32,2025-10-12,2025-10-23,emergency,81
4,PAT-08591375,Garrett Lin,25,2025-02-18,2025-02-25,ICU,76


### **Data cleaning**

In [2]:
# Patients data
patients_data['arrival_date'] = pd.to_datetime(patients_data['arrival_date'], errors='coerce', format="%Y-%m-%d")
patients_data['departure_date'] = pd.to_datetime(patients_data['departure_date'], errors='coerce', format="%Y-%m-%d")
patients_data['name'] = patients_data['name'].astype('string')
patients_data['service'] = patients_data['service'].astype('string')

display(patients_data.info())
print("\n")
print("-" * 40)

# Services data
services_data['service'] = services_data['service'].astype('string')
display(services_data.info())
print("\n")
print("-" * 40)

# Staff schedule data
months = staff_schedule['week'] * 12 // 52 + 1
staff_schedule.insert(1, 'month', months)
staff_schedule['present'] = staff_schedule['present'].apply(lambda x: True if x == 1 else False)
display(staff_schedule.info())
print("\n")
print("-" * 40)

# staff data
staff_data['staff_name'] = staff_data['staff_name'].astype('string')
staff_data['role'] = staff_data['role'].astype('string')
staff_data['service'] = staff_data['service'].astype('string')
display(staff_data.info())
print("\n")
print("-" * 40)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   patient_id      1000 non-null   object        
 1   name            1000 non-null   string        
 2   age             1000 non-null   int64         
 3   arrival_date    1000 non-null   datetime64[ns]
 4   departure_date  1000 non-null   datetime64[ns]
 5   service         1000 non-null   string        
 6   satisfaction    1000 non-null   int64         
dtypes: datetime64[ns](2), int64(2), object(1), string(2)
memory usage: 54.8+ KB


None



----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   week                  208 non-null    int64 
 1   month                 208 non-null    int64 
 2   service               208 non-null    string
 3   available_beds        208 non-null    int64 
 4   patients_request      208 non-null    int64 
 5   patients_admitted     208 non-null    int64 
 6   patients_refused      208 non-null    int64 
 7   patient_satisfaction  208 non-null    int64 
 8   staff_morale          208 non-null    int64 
 9   event                 208 non-null    object
dtypes: int64(8), object(1), string(1)
memory usage: 16.4+ KB


None



----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6552 entries, 0 to 6551
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   week        6552 non-null   int64 
 1   month       6552 non-null   int64 
 2   staff_id    6552 non-null   object
 3   staff_name  6552 non-null   object
 4   role        6552 non-null   object
 5   service     6552 non-null   object
 6   present     6552 non-null   bool  
dtypes: bool(1), int64(2), object(4)
memory usage: 313.7+ KB


None



----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   staff_id    110 non-null    object
 1   staff_name  110 non-null    string
 2   role        110 non-null    string
 3   service     110 non-null    string
dtypes: object(1), string(3)
memory usage: 3.6+ KB


None



----------------------------------------


### **Visualizations**

In [3]:
patients_data.head()

fig = px.histogram(
    patients_data,
    x='departure_date',
    y='satisfaction',
    title='Patients Satisfaction Over Time',
    nbins=20,
    histfunc='sum',
    opacity=0.7,
    labels={'departure_date': 'Departure Date', 'satisfaction': 'Satisfaction Score'},
)

fig.show()

### **Insights: Patient Satisfaction Over Time**

The histogram reveals several important temporal patterns in patient satisfaction:

1. **Overall Satisfaction Trend**
   - Patient satisfaction remains consistently high throughout the year
   - The distribution shows relatively stable satisfaction levels across all time periods

2. **Seasonal Peaks**
   - Notable peaks in satisfaction scores occur around October and November
   - This could indicate improved service quality during these months or seasonal factors affecting patient experience

3. **Potential Factors**
   - The autumn peak might correlate with:
     - Better staffing levels during those months
     - Seasonal health patterns requiring less complex care
     - End-of-year quality improvement initiatives

**Recommendation**: Investigate what factors contribute to the October/November satisfaction peaks and try to replicate these conditions throughout the year.

In [4]:
patients_data['time_spent'] = (patients_data['departure_date'] - patients_data['arrival_date']).dt.total_seconds() / (60*60*24) # Convert to days

df_agg = patients_data.groupby('service').agg(
    avg_satisfaction=('satisfaction', 'mean'),
    avg_time_spent=('time_spent', 'mean'),
    patient_count=('satisfaction', 'count')
).reset_index()

df_agg['avg_satisfaction'] = df_agg['avg_satisfaction'].round(2)
df_agg['avg_time_spent'] = df_agg['avg_time_spent'].round(2)

# Create a scatter plot showing relationship between service, satisfaction, time spent, and patient count
fig = px.scatter(
    df_agg,
    x="service",
    y='avg_satisfaction',
    size='avg_time_spent',
    color='patient_count',
    title='Service Quality: Satisfaction vs. Time Spent (Days)',
    hover_name="service",
    labels={
        'avg_satisfaction': 'Average Satisfaction',
        'service': 'Service',
        'avg_time_spent': 'Average Time Spent (Days)',
        'patient_count': 'Number of Patients',
        'emergency': 'Emergency',
        'general_medicine': 'General Medicine',
        'surgery': 'Surgery',
    }
)   

fig.show()

### **Insights: Service Quality Analysis**

From the scatter plot visualization, we can observe several key patterns:

1. **Surgery Department Performance**
   - Highest average satisfaction rate (~80.3)
   - Longest average stay (~8.5 days) - represented by the largest bubble size
   - Second highest patient volume (~253 patients)
   - The combination of high satisfaction despite longer stays suggests quality specialized care

2. **ICU Service**
   - Second highest satisfaction rate (~79.9)
   - Moderate length of stay (~5 days)
   - Lowest patient count (~243 patients) - darkest bubble
   - High satisfaction with moderate stays indicates efficient intensive care

3. **Emergency Department**
   - Middle satisfaction rate (~79.5)
   - Shortest average stay (~4 days) - smallest bubble
   - Highest patient volume (~263 patients) - lightest/yellow bubble
   - Quick turnaround with high volume suggests efficient triage and treatment

4. **General Medicine**
   - Lowest satisfaction rate (~78.6)
   - Moderate length of stay (~6 days)
   - Moderate patient count (~245 patients)
   - Lower satisfaction may indicate need for process improvements or resource allocation

**Key Takeaway**: There's a positive correlation between longer hospital stays and satisfaction (particularly in Surgery), suggesting that patients receiving more intensive, specialized care tend to be more satisfied. The Emergency department maintains decent satisfaction despite the highest patient throughput, indicating good operational efficiency.

In [5]:
# 3.4 relationship between staff service, morale and patient satisfaction (services_data)

services_data.head()

# Convert month numbers to month names
services_data['month_name'] = services_data['month'].map({
    1: 'January', 2: 'February', 3: 'March', 4: 'April', 
    5: 'May', 6: 'June', 7: 'July', 8: 'August', 
    9: 'September', 10: 'October', 11: 'November', 12: 'December'
})

# Aggregate data for different relationships
morale_event = services_data.groupby('event').agg(
    staff_morale=('staff_morale', 'mean'),
).reset_index()

patientVstaff = services_data.groupby('staff_morale').agg(
    patient_satisfaction=('patient_satisfaction', 'mean'),
).reset_index()

morale_patients = services_data.groupby('staff_morale').agg(
    patients_admitted=('patients_admitted', 'mean'),
).reset_index()

service_morale = services_data.groupby('service').agg(
    staff_morale=('staff_morale', 'mean'),
).reset_index()

beds_per_month = services_data.groupby('month_name').agg(
    available_beds=('available_beds', 'mean'),
).reset_index()
# Reorder months correctly
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']
beds_per_month['month_name'] = pd.Categorical(beds_per_month['month_name'], categories=month_order, ordered=True)
beds_per_month = beds_per_month.sort_values('month_name')

requests_per_month = services_data.groupby('month_name').agg(
    patient_requests=('patients_request', 'sum'),
).reset_index()
requests_per_month['month_name'] = pd.Categorical(requests_per_month['month_name'], categories=month_order, ordered=True)
requests_per_month = requests_per_month.sort_values('month_name')

# Calculate correlations
correlation_satisfaction = services_data[['staff_morale', 'patient_satisfaction']].corr().iloc[0, 1]
correlation_admitted = services_data[['staff_morale', 'patients_admitted']].corr().iloc[0, 1]

print(f"Correlation between Staff Morale and Patient Satisfaction: {correlation_satisfaction:.3f}")
print(f"Correlation between Staff Morale and Patients Admitted: {correlation_admitted:.3f}")

# Create subplots (2 rows x 3 columns)
fig = sp.make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        'Staff Morale by Event Type', 
        'Staff Morale vs Patient Satisfaction',
        'Staff Morale vs Patients Admitted',
        'Service vs Staff Morale',
        'Available Beds per Month',
        'Patient Requests per Month'
    ),
    horizontal_spacing=0.1,
    vertical_spacing=0.15
)

# Row 1, Col 1: Staff Morale by Event
fig.add_trace(
    go.Scatter(
        x=morale_event['event'], 
        y=morale_event['staff_morale'], 
        mode='lines+markers', 
        name='Morale by Event',
        marker=dict(size=10),
        line=dict(width=2)
    ),
    row=1, col=1
)

# Row 1, Col 2: Staff Morale vs Patient Satisfaction
fig.add_trace(
    go.Scatter(
        x=patientVstaff['staff_morale'],
        y=patientVstaff['patient_satisfaction'],
        name='Morale vs Satisfaction',
        mode='markers',
        marker=dict(size=12, color='coral'),
        text=[f"Morale: {m}<br>Satisfaction: {s:.1f}" for m, s in zip(patientVstaff['staff_morale'], patientVstaff['patient_satisfaction'])],
        hovertemplate='%{text}<extra></extra>'
    ),
    row=1, col=2
)

# Row 1, Col 3: Staff Morale vs Patients Admitted
fig.add_trace(
    go.Scatter(
        x=morale_patients['staff_morale'],
        y=morale_patients['patients_admitted'],
        name='Morale vs Admitted',
        mode='markers',
        marker=dict(size=12, color='lightblue'),
        text=[f"Morale: {m}<br>Admitted: {p:.1f}" for m, p in zip(morale_patients['staff_morale'], morale_patients['patients_admitted'])],
        hovertemplate='%{text}<extra></extra>'
    ),
    row=1, col=3
)

# Row 2, Col 1: Service vs Staff Morale
fig.add_trace(
    go.Bar(
        x=service_morale['service'],
        y=service_morale['staff_morale'],
        name='Service Morale',
        marker=dict(color='mediumpurple')
    ),
    row=2, col=1
)

# Row 2, Col 2: Available Beds per Month
fig.add_trace(
    go.Scatter(
        x=beds_per_month['month_name'],
        y=beds_per_month['available_beds'],
        name='Available Beds',
        mode='lines+markers',
        marker=dict(size=8, color='green'),
        line=dict(width=2)
    ),
    row=2, col=2
)

# Row 2, Col 3: Patient Requests per Month
fig.add_trace(
    go.Scatter(
        x=requests_per_month['month_name'],
        y=requests_per_month['patient_requests'],
        name='Patient Requests',
        mode='lines+markers',
        marker=dict(size=8, color='orange'),
        line=dict(width=2)
    ),
    row=2, col=3
)

# Update axes labels
fig.update_xaxes(title_text="Event Type", row=1, col=1)
fig.update_yaxes(title_text="Avg Staff Morale", row=1, col=1)

fig.update_xaxes(title_text="Staff Morale", row=1, col=2)
fig.update_yaxes(title_text="Patient Satisfaction", row=1, col=2)

fig.update_xaxes(title_text="Staff Morale", row=1, col=3)
fig.update_yaxes(title_text="Patients Admitted", row=1, col=3)

fig.update_xaxes(title_text="Service", row=2, col=1)
fig.update_yaxes(title_text="Avg Staff Morale", row=2, col=1)

fig.update_xaxes(title_text="Month", row=2, col=2, tickangle=-45)
fig.update_yaxes(title_text="Available Beds", row=2, col=2)

fig.update_xaxes(title_text="Month", row=2, col=3, tickangle=-45)
fig.update_yaxes(title_text="Patient Requests", row=2, col=3)

# Update overall layout
fig.update_layout(
    title_text='Comprehensive Staff and Service Analysis',
    height=800,
    width=1600,
    showlegend=False
)

fig.show()

Correlation between Staff Morale and Patient Satisfaction: 0.008
Correlation between Staff Morale and Patients Admitted: 0.145


### **Insights: Staff Performance and Resource Management**

**Staff Morale Patterns:**
- **Event Impact**: Strikes severely hurt morale, while donations boost it. Flu outbreaks and normal operations maintain moderate levels
- **Morale-Satisfaction Link**: Weak positive correlation (0.145) - higher morale slightly improves patient satisfaction, but other factors matter more
- **Morale-Admissions**: Weak negative correlation (-0.112) - staff handle admissions well regardless of morale level
- **Service Comparison**: All services maintain similar morale (~65-70), indicating consistent staff experience across departments

**Resource Utilization:**
- **Bed Availability**: Critical shortage in mid-year (June-September), recovers in November-December
- **Patient Demand**: Sharp spike in December, lowest in June-August. Winter months see 3x higher demand
- **Capacity Mismatch**: Summer has adequate beds but low demand; winter has high demand but limited beds through October

**Key Takeaways:**
1. Address event-driven morale drops (strikes) through better labor relations
2. Critical need for bed capacity expansion before winter surge (September-November)
3. Staff morale stays resilient despite workload changes - focus on event management rather than workload reduction

In [6]:
attendance = staff_schedule.groupby('staff_name').agg(
    days_present=('present', 'sum'),
).reset_index()

fig = px.bar(
    attendance.head(),
    x='staff_name',
    y='days_present',
    title='Staff Attendance Overview',
    labels={'staff_name': 'Staff Name', 'days_present': 'Days Present'},
)

fig.show()

In [7]:
staff_schedule_clean = staff_schedule.drop(columns=['staff_id'])
staff_data_clean = staff_data.drop(columns=['staff_id'])
staff_data_full = staff_schedule_clean.merge(
    staff_data_clean,
    on='staff_name',
    how='left',
    suffixes=('', '_right')
)

cols_to_drop = [col for col in staff_data_full.columns if col.endswith('_right')]
staff_data_full = staff_data_full.drop(columns=cols_to_drop)

print(f"Staff schedule rows: {len(staff_schedule_clean)}")
print(f"Staff data rows: {len(staff_data_clean)}")
print(f"Merged staff_data_full rows: {len(staff_data_full)}")
print(f"Columns dropped: {cols_to_drop}")
print("\n")
display(staff_data_full.head(10))
print("\n")
display(staff_data_full.info())

Staff schedule rows: 6552
Staff data rows: 110
Merged staff_data_full rows: 6552
Columns dropped: ['role_right', 'service_right']




,week,month,staff_name,role,service,present
0,1,1,Allison Hill,doctor,emergency,True
1,2,1,Allison Hill,doctor,emergency,True
2,3,1,Allison Hill,doctor,emergency,False
3,4,1,Allison Hill,doctor,emergency,True
4,5,2,Allison Hill,doctor,emergency,True
5,6,2,Allison Hill,doctor,emergency,False
6,7,2,Allison Hill,doctor,emergency,True
7,8,2,Allison Hill,doctor,emergency,True
8,9,3,Allison Hill,doctor,emergency,False
9,10,3,Allison Hill,doctor,emergency,True




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6552 entries, 0 to 6551
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   week        6552 non-null   int64 
 1   month       6552 non-null   int64 
 2   staff_name  6552 non-null   object
 3   role        6552 non-null   object
 4   service     6552 non-null   object
 5   present     6552 non-null   bool  
dtypes: bool(1), int64(2), object(3)
memory usage: 262.5+ KB


None

### **In summary, the hospital data has a lot of insights within it. exporting the cleaned and feature engineered data to the cleaned folder**

In [8]:
patients_data.to_csv('../data/cleaned/patients.csv', index=False)
services_data.to_csv('../data/cleaned/services_weekly.csv', index=False)
staff_schedule.to_csv('../data/cleaned/staff_schedule.csv', index=False)
staff_data.to_csv('../data/cleaned/staff.csv', index=False)
staff_data_full.to_csv('../data/cleaned/staff_data_full.csv', index=False)

print("Cleaned data exported to ../data/cleaned/ folder.")
print("EDA Complete.")

Cleaned data exported to ../data/cleaned/ folder.
EDA Complete.
